In [ ]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.chains import RetrievalQA
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai.llms import GoogleGenerativeAI
from langchain.vectorstores import FAISS
import pinecone
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

e:\LangChain\LangChain\End-End-GenAI-Education-Industry-Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [6]:

# Configure Google Generative AI
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise EnvironmentError("GOOGLE_API_KEY not found. Please set it in your .env file.")

genai.configure(api_key=GOOGLE_API_KEY)

# Initialize Language Model
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash', google_api_key=GOOGLE_API_KEY)

# Load csv file

In [6]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='questions.csv',source_column='prompt')
dataset = loader.load()


# GoogleGenerativeAI for Word Embedding

In [7]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
e = embeddings.embed_query("What is your refund policy?")
type(e)
len(e)

768

# Create Vector DB

In [8]:
# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=dataset,
                                 embedding=embeddings)

In [9]:
# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.3)
docs = retriever.get_relevant_documents("duration of bootcamp?")
print(docs)

C:\Users\Dell\AppData\Local\Temp\ipykernel_9408\1824494725.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("duration of bootcamp?")


[Document(metadata={'source': 'smart_shopping_assistant.csv', 'row': 604}, page_content='Product Name: Product 605\nCategory: Toys\nPrice: $6100\nAvailability: In Stock\nDescription: Description for Product 605'), Document(metadata={'source': 'smart_shopping_assistant.csv', 'row': 559}, page_content='Product Name: Product 560\nCategory: Toys\nPrice: $5650\nAvailability: Out of Stock\nDescription: Description for Product 560'), Document(metadata={'source': 'smart_shopping_assistant.csv', 'row': 464}, page_content='Product Name: Product 465\nCategory: Toys\nPrice: $4700\nAvailability: In Stock\nDescription: Description for Product 465'), Document(metadata={'source': 'smart_shopping_assistant.csv', 'row': 564}, page_content='Product Name: Product 565\nCategory: Toys\nPrice: $5700\nAvailability: In Stock\nDescription: Description for Product 565')]


# Retrievel QA

In [10]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff',retriever=retriever,input_key='query',return_source_documents=True)

# Q&A

In [15]:
#print(chain("What if I don’t like this bootcamp?"))
#print(chain('What is the tuition fee for undergraduate programs?'))
print(chain('Product5'))

{'query': 'Product5', 'result': 'Product 5 is a toy that costs $100 and is currently in stock.\n', 'source_documents': [Document(metadata={'source': 'smart_shopping_assistant.csv', 'row': 50}, page_content='Product Name: Product 51\nCategory: Electronics\nPrice: $560\nAvailability: In Stock\nDescription: Description for Product 51'), Document(metadata={'source': 'smart_shopping_assistant.csv', 'row': 4}, page_content='Product Name: Product 5\nCategory: Toys\nPrice: $100\nAvailability: In Stock\nDescription: Description for Product 5'), Document(metadata={'source': 'smart_shopping_assistant.csv', 'row': 550}, page_content='Product Name: Product 551\nCategory: Electronics\nPrice: $5560\nAvailability: In Stock\nDescription: Description for Product 551'), Document(metadata={'source': 'smart_shopping_assistant.csv', 'row': 52}, page_content='Product Name: Product 53\nCategory: Home Appliances\nPrice: $580\nAvailability: In Stock\nDescription: Description for Product 53')]}


In [ ]:
# print(chain("""I have zero knowledge of Excel and belong to a non-technical
#  background. Can I take this course?"""))

{'query': 'I have zero knowledge of Excel and belong to a non-technical\n background. Can I take this course?', 'result': "I don't know.  The provided text is about product information and doesn't contain information about a course or its prerequisites.\n", 'source_documents': [Document(metadata={'source': 'smart_shopping_assistant.csv', 'row': 783}, page_content='Product Name: Product 784\nCategory: Books\nPrice: $7890\nAvailability: Out of Stock\nDescription: Description for Product 784'), Document(metadata={'source': 'smart_shopping_assistant.csv', 'row': 873}, page_content='Product Name: Product 874\nCategory: Books\nPrice: $8790\nAvailability: Out of Stock\nDescription: Description for Product 874'), Document(metadata={'source': 'smart_shopping_assistant.csv', 'row': 883}, page_content='Product Name: Product 884\nCategory: Books\nPrice: $8890\nAvailability: Out of Stock\nDescription: Description for Product 884'), Document(metadata={'source': 'smart_shopping_assistant.csv', 'row':